In [1]:
import argparse
import pickle
from pathlib import Path
import sys
import wandb

In [2]:
PROJECT_PATH = "/ziob/klaudia/FlowHMM"
sys.path.insert(1, PROJECT_PATH)

In [3]:
from clickstream_experiment.source.tagnn.models.tagnn.model import SessionGraph
from clickstream_experiment.source.tagnn.models.tagnn.wrappers import Trainer
from clickstream_experiment.source.tagnn.models.tagnn.utils import (
    Data, trans_to_cuda, find_edge_weights, read_data, calculate_embeddings, calculate_abx, split_validation
)

In [4]:
# parser.add_argument('--data_path', default='/Ziob/adurb/data/yoochoose/processed')
# parser.add_argument('--dataset', default='yoochoose1_64', help='dataset name: diginetica/yoochoose1_4/yoochoose1_64/sample')
# parser.add_argument('--batchSize', type=int, default=32, help='input batch size')
# parser.add_argument('--hiddenSize', type=int, default=100, help='hidden state size')
# parser.add_argument('--epoch', type=int, default=30, help='the number of epochs to train for')
# parser.add_argument('--lr', type=float, default=0.001, help='learning rate')  # [0.001, 0.0005, 0.0001]
# parser.add_argument('--lr_dc', type=float, default=0.1, help='learning rate decay rate')
# parser.add_argument('--lr_dc_step', type=int, default=3, help='the number of steps after which the learning rate decay')
# parser.add_argument('--l2', type=float, default=1e-5, help='l2 penalty')  # [0.001, 0.0005, 0.0001, 0.00005, 0.00001]
# parser.add_argument('--step', type=int, default=1, help='gnn propogation steps')
# parser.add_argument('--patience', type=int, default=3, help='the number of epoch to wait before early stop ')
# parser.add_argument('--variant', default='hybrid')
# parser.add_argument('--ignore_target', action='store_true')
# parser.add_argument('--validation', action='store_true', help='validation')
# parser.add_argument('--valid_portion', type=float, default=0.1, help='split the portion of training set as validation set')
# parser.add_argument('--device_ids', type=int, nargs='*')
# parser.add_argument('--init_embeddings', action='store_true', help='initialize embeddings using word2vec')
# parser.add_argument('--unfreeze_embeddings', type=int, default=1, help='epoch in which to unfreeze the embeddings layer')
# parser.add_argument('--name', type=str, help='name of wandb run')
# parser.add_argument('--use_global_graph', action='store_true')
# parser.add_argument('--log_freq', type=int, default=5, help='how many times to log during an epoch')
# parser.add_argument('--results_path', type=str, help='file to save model results in')
# parser.add_argument('--model_dir', type=str, help='directory to save model weights in')
opt = dict(data_path='/ziob/adurb/data/yoochoose/processed', dataset='yoochoose1_64', name=None, validation=True, valid_portion=0.1, batchSize=32, hiddenSize=100, epoch=2, lr=0.01, lr_dc=.1, lr_dc_step=3, l2=1e-5, step=1, patience=2, variant='hybrid', ignore_target=False, log_freq=5,  init_embeddings=False, model_dir='/ziob/klaudia/FlowHMM/clickstream_experiment/logs', results_path='/ziob/klaudia/FlowHMM/clickstream_experiment/logs')

In [5]:
data_path = Path(opt['data_path'])

In [6]:
dataset_version = data_path.parts[-1]

In [7]:
name = f"TAGNN_{opt['dataset']}_{dataset_version}" if opt['name'] is None else opt['name']

In [8]:
import pandas as pd
import numpy as np
from clickstream_experiment.source.tagnn.utils.utils import normalize, flatten  # TODO

In [49]:
data_path, dataset, validation, valid_portion = data_path, opt['dataset'], opt['validation'], opt['valid_portion']
train_data = pickle.load(open(data_path / f"{data_path}/clicks_train_1_64.pkl", 'rb'))
if validation:
    train_data, valid_data = split_validation(train_data, valid_portion, seed=42)
    test_data = valid_data
else:
    test_data = pickle.load(open(data_path / f"{dataset}/test.txt", 'rb'))

clicks_pdf = pd.read_parquet(
    data_path / 'clicks_train_1_64.parquet',
    # header=None,
    # names=['sessionID', 'timestamp', 'itemID', 'category'],
    # parse_dates=['timestamp'],
    # dtype={'sessionID': int, 'itemID': int, 'category': str},
)

# item2id = pickle.load(open(data_path / "item2id.txt", 'rb'))
# id2item = {new_id: item_id for item_id, new_id in item2id.items()}
# train_data = pickle.load(open(data_path / "yoochoose1_64/train.txt", 'rb'))

# dates = pickle.load(open(data_path / "yoochoose1_64/dates.txt", 'rb'))
# dates = pd.to_datetime(dates, unit='s')
# if dates.tz is None:
#     dates = dates.tz_localize('UTC')
# max_date = max(dates)

items_in_train = np.unique(flatten(train_data)).astype('int64')

id2item = {i: items_in_train[i] for i in range(len(items_in_train))}
item2id = {items_in_train[i]: i for i in range(len(items_in_train))}

filtered_clicks_pdf = clicks_pdf # [clicks_pdf['timestamp'] < max_date]

long_sessions = filtered_clicks_pdf['sessionID'].value_counts() >= 2
long_sessions = long_sessions.index[long_sessions]

filtered_clicks_pdf = filtered_clicks_pdf[filtered_clicks_pdf['sessionID'].isin(long_sessions)]

train_data, test_data, filtered_clicks_pdf, items_in_train #, item2id


(([[4078, 36674],
   [35583, 2461, 26898, 2461, 35583, 2461, 35713],
   [22090, 35586, 342],
   [335, 335],
   [35709, 35710],
   [335, 3998, 5407, 7158, 4976, 4005, 5192, 35533, 3998],
   [21386, 21383],
   [335, 35601, 35601, 35601],
   [36543, 36546, 36545, 35645],
   [36587, 17672],
   [35619, 35629, 35636, 35484],
   [17354],
   [35145, 13085, 13087],
   [2973],
   [17799],
   [36097, 37227, 37228, 37224],
   [21192, 34742, 21192, 34163, 22108, 23346, 35619],
   [36418,
    12217,
    2655,
    29616,
    29617,
    29615,
    29614,
    33001,
    20806,
    31350,
    12216,
    35090,
    21679,
    35350,
    34852,
    34853,
    34854,
    34855,
    34856,
    34857,
    34858,
    34859,
    34860,
    34861,
    22072,
    23037,
    20249,
    35364,
    35366,
    36946,
    36947,
    8529,
    8160,
    35351,
    35352,
    7514,
    4718,
    18548,
    22606,
    20204,
    26964,
    24205,
    22537,
    34623,
    4363,
    22862,
    34771,
    28160,
    18846

In [35]:
n_node = items_in_train.shape[0]

In [46]:
test_data[1].index(4682)

97

In [50]:
train_data = [[item2id[i] for i in s] for s in train_data[0]], [item2id[i] for i in train_data[1]]
test_data  = [[item2id[i] for i in s] for s in test_data[0]],  [item2id[i] for i in test_data[1]]

KeyError: 4682

In [26]:
model = SessionGraph(opt, n_node, init_embeddings=None)
model = trans_to_cuda(model).float()

trainer = Trainer(model, log_freq=opt['log_freq'], model_dir=opt['model_dir'])
unfreeze_embeddings = None
trainer.fit(Data(train_data), Data(test_data), opt['epoch'], opt['patience'], unfreeze_embeddings, 0.1)

-------------------------------------------------------
epoch:  0


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
train_data

In [14]:
print(items_in_train)


[    3     4     5 ... 37473 37474 37475]
